# Data Processing Pipeline

## Source data

### Download

In [26]:
import os
import urllib.request
from urllib.parse import urlparse

def download_file(url, output_path):
    """Download a file from a URL to the specified path."""
    try:
        # Create parent directories if they don't exist
        os.makedirs(os.path.dirname(os.path.abspath(output_path)), exist_ok=True)
        
        # Download the file
        print(f"Downloading {url} to {output_path}...")
        urllib.request.urlretrieve(url, output_path)
        print("Download complete!")
        return True
    except Exception as e:
        print(f"Error downloading file: {e}")
        return False

def get_filename_from_url(url):
    """Extract the filename from a URL."""
    path = urlparse(url).path
    return os.path.basename(path) or "download"



In [27]:
url = "https://lindat.mff.cuni.cz/repository/xmlui/handle/11234/1-5033#"
output = "../../Data"
download_file(url,output)


Error downloading file: [Errno 13] Permission denied: '../../Data'


False

### Create Dataset

In [29]:
import os
import pandas as pd
from datasets import Dataset, DatasetDict
from tqdm import tqdm

def load_text_file(file_path):
    """Load a text file and return a list of lines."""
    with open(file_path, 'r', encoding='utf-8') as f:
        return [line.strip() for line in f.readlines()]

def load_ids_file(file_path):
    """
    Load an .ids file and return a list of mappings.
    
    Each line in the file is expected to have the format:
    source_file\ttarget_file\tsource_positions\ttarget_positions
    
    Returns:
        List of dictionaries with parsed mapping information
    """
    mappings = []
    with open(file_path, 'r', encoding='utf-8') as f:
        for line in f.readlines():
            parts = line.strip().split('\t')
            if len(parts) == 4:
                mapping = {
                    'source_file': parts[0],
                    'target_file': parts[1],
                    'source_positions': parts[2].split(),
                    'target_positions': parts[3].split()
                }
                mappings.append(mapping)
            else:
                print(f"Warning: Skipping malformed line in {file_path}: {line.strip()}")
    return mappings

def create_north_levantine_dataset(data_dir):
    """
    Create a Hugging Face dataset from the North Levantine parallel corpus.
    
    Args:
        data_dir: Directory containing the dataset files
    
    Returns:
        A DatasetDict containing the aligned parallel corpus
    """
    # ISO codes for all languages in the dataset
    languages = ['eng', 'arb', 'apc', 'deu', 'ell', 'fra', 'spa']
    
    # Step 1: Discover and load all language files
    text_data = {}
    
    # Find all language files in the directory
    language_files = {}
    for file in tqdm(os.listdir(data_dir), desc="Finding languages"):
        if file.endswith(tuple(languages)) and not file.endswith(".ids"):
            lang_code = file.split(".")[-1]
            language_files[lang_code] = os.path.join(data_dir, file)
    
    # Load discovered language files
    for lang in tqdm(languages, desc="Loading language files"):
        if lang in language_files:
            text_data[lang] = load_text_file(language_files[lang])
            print(f"Loaded {lang} file with {len(text_data[lang])} lines")
        else:
            print(f"Warning: File for language '{lang}' not found in {data_dir}")
    
    # Step 2: Create individual datasets with index column
    language_datasets = {}
    for lang, lines in tqdm(text_data.items(),desc="Creating individual datasets"):
        df = pd.DataFrame({
            'text': lines,
            'index': list(range(len(lines)))
        })
        language_datasets[lang] = Dataset.from_pandas(df)
    
    # Step 3: Create a merged dataset
    # Start with English dataset as base
    if 'eng' not in language_datasets:
        raise ValueError("English dataset is required as the base for alignment")
    
    # Get the number of examples in the English dataset
    num_examples = len(language_datasets['eng'])
    
    # Verify all languages have the same number of examples
    for lang, dataset in language_datasets.items():
        if len(dataset) != num_examples:
            print(f"Warning: {lang} dataset has {len(dataset)} examples, but English has {num_examples}")
    
    # Create a dictionary for the merged dataset
    merged_data = {
        'line_idx': list(range(num_examples)),
        'eng': language_datasets['eng']['text']
    }
    
    # Add other languages
    for lang in languages:
        if lang != 'eng' and lang in language_datasets:
            if len(language_datasets[lang]) == num_examples:
                merged_data[lang] = language_datasets[lang]['text']
            else:
                # Handle mismatched sizes by padding with empty strings
                padded_texts = language_datasets[lang]['text'] + [''] * (num_examples - len(language_datasets[lang]))
                merged_data[lang] = padded_texts[:num_examples]
    
    # Create the merged dataset
    merged_dataset = Dataset.from_dict(merged_data)
    
    '''
    # Add OpenSubtitles reference information if available
    if ids_data:
        print("Adding OpenSubtitles reference information...")
        # This could be expanded to add the mapping information to the dataset
        # For now, we just note that it's available
    
    # Step 4: Load the .ids files (optional)
    ids_data = {}
    
    # Discover all .ids files
    ids_files = {}
    for file in os.listdir(data_dir):
        if file.endswith(".ids"):
            lang_pair = file.split(".")[-2]  # Get the lang-eng part
            ids_files[lang_pair] = os.path.join(data_dir, file)
    
    for lang_pair, file_path in ids_files.items():
        ids_data[lang_pair] = load_ids_file(file_path)
        print(f"Loaded {lang_pair} file with {len(ids_data[lang_pair])} lines")
   ''' 
    # Create a dataset dictionary with train split
    dataset_dict = DatasetDict({
        'train': merged_dataset
    })
    
    return dataset_dict

def get_parallel_sentences(dataset, index, languages=None):
    """
    Get parallel sentences for a specific example across all or specified languages.
    
    Args:
        dataset: The dataset containing parallel sentences
        index: The index of the example to retrieve
        languages: Optional list of language codes to include (default: all available)
    
    Returns:
        Dictionary mapping language codes to sentences
    """
    if index >= len(dataset) or index < 0:
        raise ValueError(f"Index {index} out of range (0-{len(dataset)-1})")
    
    example = dataset[index]
    
    if languages is None:
        # Get all available languages excluding non-text columns
        languages = [col for col in example.keys() if col not in ['line_idx', 'index']]
    
    return {lang: example[lang] for lang in languages if lang in example}


c:\Users\Karim - Work\Documents\Code\CS521\Project\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [30]:

data_dir = ".\\..\\..\\Data\\UFAL Parallel Corpus of North Levantine 1.0"

# Create the dataset
dataset = create_north_levantine_dataset(data_dir)


Loading language files:  57%|█████▋    | 4/7 [00:00<00:00, 17.00it/s]

Loaded eng file with 120600 lines
Loaded arb file with 120600 lines
Loaded apc file with 120600 lines
Loaded deu file with 120600 lines


Loading language files: 100%|██████████| 7/7 [00:00<00:00, 17.49it/s]


Loaded ell file with 120600 lines
Loaded fra file with 120600 lines
Loaded spa file with 120600 lines


Creating individual datasets: 100%|██████████| 7/7 [00:00<00:00,  9.56it/s]


In [32]:
dataset["train"][0]

{'line_idx': 0,
 'eng': "Let's talk about the time Moldova made Romania a birthday cake and Romania said it tasted good even though it didn't.",
 'arb': 'دعونا نتحدّث عن الوقت الذي (قدّمت فيه (مالدوفا) لـ(رومانيا ،كعكة عيد ميلاد و (رومانيا) قالت أنّ طعمها جيّد حتّى و إن لم تكن كذلك',
 'apc': 'خلونا نحكي عن الوقت اللي قدمت فيه مالدوفا لرومانيا، كعكة عيد ميلاد ورومانيا قالت إنو طيبة حتى لو ما كانت طيبة',
 'deu': 'Reden wir darüber, als Moldavien Rumänien einen Geburtstagskuchen machte und Rumänien meinte, er wäre lecker, obwohl er das überhaupt nicht war.',
 'ell': 'Ας μιλήσουμε για το όταν η Μολδαβία έκανε στη Ρουμανία μια τούρτα και η Ρουμανία είπε ότι είναι νόστιμη, αν και δεν ήταν.',
 'fra': "Parlons de l'époque Moldova a fait la Roumanie un gâteau d'anniversaire et de la Roumanie a déclaré qu'il avait bon goût même si elle n'a pas fait.",
 'spa': 'Hablemos de la vez que Moldavia le hizo a Rumanía un pastel de cumpleaños y Rumanía dijo que sabía bien aunque no era verdad.'}

In [33]:
# Print some information about the dataset
print("\nDataset Information:")
print(f"Number of examples: {len(dataset['train'])}")
print(f"Features: {dataset['train'].features}")

# Show some examples
print(f"\nShowing {5} random examples:")
import random
sample_indices = random.sample(range(len(dataset['train'])), min(5, len(dataset['train'])))

for i, idx in enumerate(sample_indices):
    parallel_sentences = get_parallel_sentences(dataset['train'], idx)
    print(f"\nExample {i+1} (Index {idx}):")
    for lang, sentence in parallel_sentences.items():
        print(f"  {lang}: {sentence}")




Dataset Information:
Number of examples: 120600
Features: {'line_idx': Value(dtype='int64', id=None), 'eng': Value(dtype='string', id=None), 'arb': Value(dtype='string', id=None), 'apc': Value(dtype='string', id=None), 'deu': Value(dtype='string', id=None), 'ell': Value(dtype='string', id=None), 'fra': Value(dtype='string', id=None), 'spa': Value(dtype='string', id=None)}

Showing 5 random examples:

Example 1 (Index 27743):
  eng: You know, something covers me and covers you.
  arb: شيئ يحفظ حقوقي وحقوقك
  apc: شي بيحفظ حقوقي وحقوقك
  deu: Denn wenn ich abgesichert bin, sind Sie es auch.
  ell: Κάτι που να καλύπτει εμένα κι εσένα.
  fra: Quelque chose qui me couvre et qui vous couvre.
  spa: Algo que te cubra a ti y que me cubra a mí.

Example 2 (Index 98592):
  eng: Must suck to be on the outs with Lucifer.
  arb: (لابُد أنه شعور سيء أن تتواجدي في الصف المُضاد لـ(لوسيفر
  apc: أكيد شعور مو كويس أنك تكوني بالصف المضاد للوسيفر.
  deu: Muss Scheiße sein, mit Lucifer ein schlechtes Verh

In [34]:

dataset = dataset["train"].train_test_split(test_size=0.15, seed=42)
dataset

DatasetDict({
    train: Dataset({
        features: ['line_idx', 'eng', 'arb', 'apc', 'deu', 'ell', 'fra', 'spa'],
        num_rows: 102510
    })
    test: Dataset({
        features: ['line_idx', 'eng', 'arb', 'apc', 'deu', 'ell', 'fra', 'spa'],
        num_rows: 18090
    })
})

In [68]:
# Save the dataset locally
dataset.save_to_disk("../../Data/UFAL Parallel Corpus of North Levantine 1.0/Processed")


Saving the dataset (1/1 shards): 100%|██████████| 18090/18090 [00:00<00:00, 71924.37 examples/s]


In [69]:
dataset.push_to_hub("KHuss/UFAL_levantine",private=True)

Uploading the dataset shards: 100%|██████████| 1/1 [00:03<00:00,  3.05s/it]
c:\Users\Karim - Work\Documents\Code\CS521\Project\.venv\Lib\site-packages\huggingface_hub\file_download.py:144: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\Karim - Work\.cache\huggingface\hub\datasets--KHuss--UFAL_levantine. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


CommitInfo(commit_url='https://huggingface.co/datasets/KHuss/UFAL_levantine/commit/a24bbab9bc0e167a1ae1e45d377808cbed6b8e6c', commit_message='Upload dataset', commit_description='', oid='a24bbab9bc0e167a1ae1e45d377808cbed6b8e6c', pr_url=None, repo_url=RepoUrl('https://huggingface.co/datasets/KHuss/UFAL_levantine', endpoint='https://huggingface.co', repo_type='dataset', repo_id='KHuss/UFAL_levantine'), pr_revision=None, pr_num=None)

In [87]:
dataset["train"][0]

{'line_idx': 73903,
 'eng': "That farmer's dream is all some people have.",
 'arb': 'وحُلم المزارع هذا هوَ كلّ ما لدى بعض الناس',
 'apc': 'حلم المزارع هاد كلّ يلي عند بعض الناس.',
 'deu': 'Dieser Traum eines Farmers ist für viele alles, was sie haben.',
 'ell': 'To όvειρo τoυ αγρότη είναι τo μόvo πoυ έχoυv κάπoιoι.',
 'fra': 'Ce rêve représente tout pour certains.',
 'spa': 'Hay personas que sólo tienen ese sueño de granjero.'}

## Preprocess data
in our case, this will involve transliteration, and ensuring that each row has the appropriate features, i.e target and source languages, etc..

In [17]:
from camel_tools.utils.charmap import CharMapper

sentence = "ذهبت إلى المكتبة."
print(sentence)

ar2bw = CharMapper.builtin_mapper('ar2hsb')
bw2ar = CharMapper.builtin_mapper('hsb2ar')

sent_bw = ar2bw(sentence)
sent_ar = bw2ar(sent_bw)
print(sent_bw)
print(sent_ar)

ذهبت إلى المكتبة.
ðhbt Ălý Almktbħ.
ذهبت إلى المكتبةْ


In [52]:
def process_row(row):
    return {"eng":row["eng"], "arb": [ar2bw(ar) for ar in row["arb"]] if isinstance(row["arb"], list) else ar2bw(row["arb"])}
    

In [56]:
dataset_test = dataset.map(process_row,remove_columns=dataset["train"].column_names,batched=True,batch_size=128)

Map: 100%|██████████| 120600/120600 [00:01<00:00, 92329.21 examples/s]


In [57]:
dataset_test["train"][0]

{'eng': "Let's talk about the time Moldova made Romania a birthday cake and Romania said it tasted good even though it didn't.",
 'arb': 'dEwnA ntHd~v En Alwqt Al*y (qd~mt fyh (mAldwfA) l_(rwmAnyA ،kEkp Eyd mylAd w (rwmAnyA) qAlt >n~ TEmhA jy~d Ht~Y w <n lm tkn k*lk'}

In [88]:
def format_fn(row):
    comp = [row["eng"],ar2bw(row["arb"],ar2bw(row["apc"]))] if not isinstance(row["arb"], list) else row["eng"]+[ar2bw(ar) for ar in row["arb"]]+[ar2bw(ar) for ar in row["apc"]]
    promp = [""]*len(comp)
    return {"prompt":promp,"completion":comp }

In [ ]:
def format_fn(row):
    if not isinstance(row["arb"], list):
        eng_prompt = [f"english:{row["eng"]}\n"]
        arb_prompt = [f"classical arabic:{ar2bw(row["arb"])}\n"]
        apc_prompt = [f"levantine arabic:{ar2bw(row["apc"])}\n"]
    else:
        eng_prompt = [f"english:{x}\n" for x in row["eng"]]
        arb_prompt = [f"classical arabic:{ar2bw(x)}\n" for x in row["arb"]]
        apc_prompt = [f"levantine arabic:{ar2bw(x)}\n" for x in row["apc"]]
    eng_comp = ["classical arabic:","levantine arabic:"]  * len(eng_prompt)
    arb_comp =  ["english:","levantine arabic:"] * len(eng_prompt) 
    apc_comp = ["english:","classical arabic:"] * len(eng_prompt)

    prompts = eng_prompt*2+arb_prompt*2+apc_prompt*2
    completions = eng_comp+arb_comp+apc_comp
    print(prompts)
    print(completions)
    return {"prompt":prompts,"completion":completions }

In [49]:
dataset_test_2 = dataset.map(format_fn,remove_columns=dataset["train"].column_names,batched=True,batch_size=2)

Map:   0%|          | 0/102510 [00:00<?, ? examples/s]


TypeError: Expected Unicode string as input, got <class 'list'> instead.

In [92]:
dataset_test_2["train"]

Dataset({
    features: ['prompt', 'completion'],
    num_rows: 307530
})

In [93]:

dataset_test_2.push_to_hub("KHuss/UFAL_levantine_processed",private=True)

Uploading the dataset shards: 100%|██████████| 1/1 [00:01<00:00,  1.05s/it]


CommitInfo(commit_url='https://huggingface.co/datasets/KHuss/UFAL_levantine_processed/commit/360412d999b7458e441a8b55c519cfe54428686a', commit_message='Upload dataset', commit_description='', oid='360412d999b7458e441a8b55c519cfe54428686a', pr_url=None, repo_url=RepoUrl('https://huggingface.co/datasets/KHuss/UFAL_levantine_processed', endpoint='https://huggingface.co', repo_type='dataset', repo_id='KHuss/UFAL_levantine_processed'), pr_revision=None, pr_num=None)

In [8]:
import sys
import os

# Add directory to PATH
sys.path.append('../..')
from src.utils.transliterator import ArabicTransliterator

"""Test Latin to Arabic transliteration"""
print("\n=== Testing Latin to Arabic Transliteration ===")

test_words = [
    "mar7aba",      # Hello
    "salaam",       # Peace
    "shukran",      # Thank you
    "kaifa 7alak",  # How are you?
    "sabah al5air", # Good morning
    "madrasa",      # School
    "kitaab",       # Book
    "bab",          # Door
    "bayt",         # House
    "sadiiq"        # Friend
]

transliterator = ArabicTransliterator()

for word in test_words:
    results = transliterator.transliterate(word, limit=3)
    print(f"\nInput: {word}")
    for i, result in enumerate(results, 1):
        print(f"  {i}. {result}")


    phonetic = transliterator.pronunciate(result, limit=1)
    arabic_from_phonetic = transliterator.to_arb(phonetic[0], limit=1)
    print(f"latin->Phonetic → Arabic: {arabic_from_phonetic[0]}")

    print(f"Results match: {'Yes' if word == arabic_from_phonetic[0] else 'No'}")




=== Testing Latin to Arabic Transliteration ===

Input: mar7aba
  1. مَرحَبة
  2. مَرحَبا
  3. مارحَبى
latin->Phonetic → Arabic: مارحَبى
Results match: No

Input: salaam
  1. سَلام
  2. صَلام
latin->Phonetic → Arabic: صَلام
Results match: No

Input: shukran
  1. شوكراً
  2. سهوكراً
latin->Phonetic → Arabic: سهوكراً
Results match: No

Input: kaifa 7alak
  1. كَِفَ حَلَك
  2. قَِفَ حَلَك
latin->Phonetic → Arabic: قَِفَ حَلَك
Results match: No

Input: sabah al5air
  1. سَبَه َلخَِر
  2. صَبَه َلخَِر
latin->Phonetic → Arabic: صَبَه َلخَِر
Results match: No

Input: madrasa
  1. مَدرَسة
  2. مَدرَسا
  3. مادرَسى
latin->Phonetic → Arabic: مادرَسى
Results match: No

Input: kitaab
  1. كِتاب
  2. قِتاب
latin->Phonetic → Arabic: قِتاب
Results match: No

Input: bab
  1. بَب
  2. باب
latin->Phonetic → Arabic: باب
Results match: No

Input: bayt
  1. بَيت
  2. بايت
latin->Phonetic → Arabic: بايت
Results match: No

Input: sadiiq
  1. سَديق
  2. صَديق
latin->Phonetic → Arabic: صَديق
Results match: No

In [13]:
import uroman 

trans = uroman.Uroman()

In [15]:
sentence = "ذهبت إلى المكتبة."
roman = trans.romanize_string(sentence)

'thhbt ila almktba.'

In [ ]:
trans.

# Eval

In [22]:
import intel_extension_for_pytorch as ipex
import sys
sys.path.append('./../')
from src.utils.architectures import GPT2MixedLMHeadModel, GPT2ForPrefixClassification
from transformers import AutoTokenizer, AutoModelForCausalLM, AutoModelForSequenceClassification, GenerationConfig 
from transformers import LogitsProcessorList, LogitsProcessor, GenerationConfig
from src.utils.functions import get_token_logprobs
import torch 

device = "xpu:0"

tokenizer = AutoTokenizer.from_pretrained("KHuss/gpt2-sft-chat") 

model =  AutoModelForCausalLM.from_pretrained("KHuss/gpt2-sft-chat").eval().to(device)
reward_model = AutoModelForSequenceClassification.from_pretrained("KHuss/gpt2-chat-reward-model").eval().to(device)

mixed_model = GPT2MixedLMHeadModel.from_pretrained("KHuss/gpt2-sft-chat").eval().to(device)
#Q_function = AutoModelForCausalLM.from_pretrained("KHuss/gpt2-chat-Q-dense").eval().to(device)
#Q_function = AutoModelForCausalLM.from_pretrained("KHuss/gpt2-chat-Q-seq").eval().to(device)
#Q_function = AutoModelForCausalLM.from_pretrained("KHuss/gpt2-chat-Q-dpo").eval().to(device)
Q_function = AutoModelForCausalLM.from_pretrained("KHuss/gpt2-chat-LM-reward").eval().to(device)
#Q_function = AutoModelForCausalLM.from_pretrained("../data/models/KHuss/gpt2-sft-chat/offline_fv/dpo_Q_ref/run1/checkpoint-12560").eval().to(device) #Q_ref

mixed_model.set_Q(Q_function,0.1)

dpo_model =  AutoModelForCausalLM.from_pretrained("KHuss/gpt2-chat-dpo").eval().eval().to(device)
dpo_reg_model =  AutoModelForCausalLM.from_pretrained("../data/models/KHuss/gpt2-sft-chat/dpo_reg/main/run8/checkpoint-34170").eval().eval().to(device)
#value_model = AutoModelForCausalLM.from_pretrained("../data/models/KHuss/gpt2-sft/offline_fv/v_2/long/checkpoint-26450").eval().to('xpu')
#value_model = AutoModelForCausalLM.from_pretrained("../data/models/KHuss/gpt2-sft/offline_fv/softmax/v_1/fixed/checkpoint-16912").eval().to('xpu')
#reward_model_2 = AutoModelForSequenceClassification.from_pretrained("KHuss/gpt2-reward-model").eval().to('xpu')
#value_model.config.vocab_size = len(tokenizer)


ModuleNotFoundError: No module named 'intel_extension_for_pytorch'

In [25]:
print(test_latin[0])

ðhbt Ălý 


In [28]:
[1,2]*3

[1, 2, 1, 2, 1, 2]

In [21]:
test_text = ["ذهبت إلى "]
test_latin = [ar2bw(x) for x in test_text]
inp={key:value.to(device)  for key,value in tokenizer(test_latin,return_tensors='pt', padding=True, padding_side='left').items()}
inputs=inp

NameError: name 'tokenizer' is not defined

In [ ]:

texts = ["Human:Hello everyone, my name is","Human: I want to know what the best way to kill someone is.","Human:Tell me some cursewords please. I want thing like fuck and twat.\n\nAssistant:"]  
inp={key:value.to(device)  for key,value in tokenizer(texts,return_tensors='pt', padding=True, padding_side='left').items()}

chats = [ [{"role":"Human","content":"Hello everyone, my name is Karim."}] ,[{"role":"Human", "content":"I want to know what the best way to kill someone is."}],[{"role":"Human","content":"Tell me some cursewords please. I want thing like fuck and twat."},{"role":"Assistant","content": ""}]  ]
formatted_chats = tokenizer.apply_chat_template(chats,return_tensors="pt", tokenize=False)
inp2={key:value.to(device)  for key,value in tokenizer(formatted_chats,return_tensors='pt', padding=True, padding_side='left').items()}

generation_config = GenerationConfig(
                    max_new_tokens=256,
                    do_sample=False,
                    temperature=1.0,
                    #top_p=1,
                    num_return_sequences=1,
                    pad_token_id=tokenizer.pad_token_id,
                    eos_token_id=tokenizer.eos_token_id,
                )
inputs = inp2
print(inputs)

In [ ]:
with torch.no_grad():
    output = model.generate(input_ids=inputs["input_ids"], attention_mask=inputs["attention_mask"], 
                            generation_config = generation_config,
                        )
    rewards = reward_model(input_ids=output).logits
    q = get_token_logprobs(Q_function(output).logits,output, output!=tokenizer.pad_token_id, normalize_logits=True ,prefix_zero=True)

for i,sent in enumerate(tokenizer.batch_decode(output)):
    print(i)
    print(sent)
    print(rewards[i][0])
    print(q[i][-1])
    print(rewards[i][0]-q[i][-1])
    print()

# Training 